In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

In [ ]:
import glob
import os


from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
import numpy as np

Using TensorFlow backend.


In [ ]:
#write path of Normal folder
imd = ""


path=os.path.join(imd, '*g')
files=glob.glob(path)

normal=[]
for f1 in files:
    img = Image.open(f1)
    
    img = img.resize((224,224))
    np_img = np.array(img)
    
    
    normal.append([np_img , 0 ])
    
normal = np.array(normal)
len(normal)


In [ ]:
normal.ndim

2

In [ ]:
#write path of Glaucoma folder
imd1 = ""
path1=os.path.join(imd1, '*g')
files1=glob.glob(path1)

normal1=[]
for f2 in files1:
    img1 = Image.open(f2)
    
    img1 = img1.resize((224,224))
    np_img1 = np.array(img1)
    
    
    normal1.append([np_img1 , 1 ])
    
normal1 = np.array(normal1)
len(normal1)

In [ ]:
training_dataset = np.concatenate((normal,normal1))

In [ ]:

X = []
y = []
for i , j in training_dataset:
    X.append(i)
    y.append(j)

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
import tensorflow
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten , Activation , Dropout ,Conv2D , MaxPooling2D
from keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adadelta
from keras.utils import np_utils
from tensorflow.keras.layers import Conv2D, MaxPooling2D,ZeroPadding2D
from sklearn.model_selection import train_test_split, KFold

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state= 22)


In [ ]:
y_train = keras.utils.to_categorical(y_train,2)
y_test = keras.utils.to_categorical(y_test,2)

In [ ]:
model0=tensorflow.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="",#write path of saved weights of respective std. model
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=2,
)

In [ ]:
ada = Adadelta(lr=0.001)

In [ ]:
for layer in model0.layers:
    layer.trainable = False

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(model0)
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=ada, metrics=['accuracy'] )

In [ ]:
history =model.fit(X_train,y_train, batch_size=32, epochs=20, validation_data=(X_test , y_test) , shuffle=True)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
predicted=model.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#predicted=np.argmax(predicted,axis=1)
#y_t=np.argmax(y_test,axis=1)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predicted,y_t))


In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_t,predicted))

In [ ]:
confusion = metrics.confusion_matrix(y_t,predicted)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:

print(metrics.accuracy_score(y_t,predicted))

In [ ]:

print(1 - metrics.accuracy_score(y_t,predicted))

In [ ]:

print(metrics.recall_score(y_t,predicted))

In [ ]:
specificity = TN / (TN + FP)

print(specificity)

In [ ]:

print(1 - specificity)

In [ ]:

print(metrics.precision_score(y_t,predicted))

In [ ]:

fpr, tpr, thresholds = metrics.roc_curve(y_t, predicted)

plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve ')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')


In [ ]:
print(metrics.roc_auc_score(y_t, predicted))